In [1]:
import pyspark
from pyspark.sql import SQLContext

In [2]:
df_rtable = sqlContext.createDataFrame([("Green", "1", "1.3", "DescriptionONE"),
                                        ("Green", "0", "1.445", "DescriptionONE"),
                                        ("Blue", "4", "1.2", "DescriptionTWO"),
                                        ("Red", "5", "1.3", "Description THREE"),
                                        ("Yellow", "7", "1.325", "Description Four"), 
                                        ("Red", "9", "1.4", "DescriptionONE"),
                                        ("Red", "6", "1.72158", "Description THREE"),
                                        ("Blue", "5", "1", "DescriptionONE")],
                                       ("Input 1", "Input 2", "Input 3", "Description"))

In [3]:
from pyspark.sql.functions import upper
from pyspark.sql.functions import udf, col

In [4]:
def spacerem_upper(x):
    pos = x.find(' ')
    #print(pos)
    if pos != - 1:
        y = x[:pos] + x[pos+1 : ].upper()
        return y
    else:
        return x

In [5]:
spaceDeleteUDF = udf(spacerem_upper)

In [6]:
df_upper2 = df_rtable.withColumn("Description", spaceDeleteUDF("Description"))

In [ ]:
#spaceDeleteUDF = udf(lambda s: s.replace(" ", ""), StringType())

In [7]:
df_upper2.show()

+-------+-------+-------+----------------+
|Input 1|Input 2|Input 3|     Description|
+-------+-------+-------+----------------+
|  Green|      1|    1.3|  DescriptionONE|
|  Green|      0|  1.445|  DescriptionONE|
|   Blue|      4|    1.2|  DescriptionTWO|
|    Red|      5|    1.3|DescriptionTHREE|
| Yellow|      7|  1.325| DescriptionFOUR|
|    Red|      9|    1.4|  DescriptionONE|
|    Red|      6|1.72158|DescriptionTHREE|
|   Blue|      5|      1|  DescriptionONE|
+-------+-------+-------+----------------+



In [9]:
df_upper2.printSchema()

root
 |-- Input 1: string (nullable = true)
 |-- Input 2: string (nullable = true)
 |-- Input 3: string (nullable = true)
 |-- Description: string (nullable = true)



In [8]:
from pyspark.sql.functions import format_number

In [10]:

df_upper3 =df_upper2.select(df_upper2['Input 1'], df_upper2['Input 2'], df_upper2['Input 3'].cast('float'), df_upper2['Description'])
df_upper3.printSchema()


root
 |-- Input 1: string (nullable = true)
 |-- Input 2: string (nullable = true)
 |-- Input 3: float (nullable = true)
 |-- Description: string (nullable = true)



In [11]:
df_upper4 = df_upper3.withColumn('Input 3',format_number(df_upper3['Input 3'],4))

In [12]:
df_upper4.show()

+-------+-------+-------+----------------+
|Input 1|Input 2|Input 3|     Description|
+-------+-------+-------+----------------+
|  Green|      1| 1.3000|  DescriptionONE|
|  Green|      0| 1.4450|  DescriptionONE|
|   Blue|      4| 1.2000|  DescriptionTWO|
|    Red|      5| 1.3000|DescriptionTHREE|
| Yellow|      7| 1.3250| DescriptionFOUR|
|    Red|      9| 1.4000|  DescriptionONE|
|    Red|      6| 1.7216|DescriptionTHREE|
|   Blue|      5| 1.0000|  DescriptionONE|
+-------+-------+-------+----------------+



In [13]:
df_rtable2 = sqlContext.createDataFrame([("Green", "Night"),
                                        ("Yellow", "Morning"),
                                        ("Red", "Afternoon"),
                                        ("Blue","Evening")])

In [14]:
df_rtable4 = df_rtable2.withColumnRenamed('_1','Input 1').withColumnRenamed('_2','Day Period')
#df_rtable4 = df_rtable3.withColumnRenamed('_2','Day Period')

In [20]:
leftjoin = df_upper4.join(df_rtable4,['Input 1'],"left")

In [21]:
leftjoin.show()

+-------+-------+-------+----------------+----------+
|Input 1|Input 2|Input 3|     Description|Day Period|
+-------+-------+-------+----------------+----------+
|   Blue|      4| 1.2000|  DescriptionTWO|   Evening|
|   Blue|      5| 1.0000|  DescriptionONE|   Evening|
|  Green|      1| 1.3000|  DescriptionONE|     Night|
|  Green|      0| 1.4450|  DescriptionONE|     Night|
| Yellow|      7| 1.3250| DescriptionFOUR|   Morning|
|    Red|      5| 1.3000|DescriptionTHREE| Afternoon|
|    Red|      9| 1.4000|  DescriptionONE| Afternoon|
|    Red|      6| 1.7216|DescriptionTHREE| Afternoon|
+-------+-------+-------+----------------+----------+



In [23]:
from pyspark.sql.functions import date_format
from pyspark.sql.functions import lit
import random

In [26]:
from pyspark.sql.types import *

In [36]:
def date_rand(a):
    date_YYYY = str(random.randint(1950, 2000))
    date_MM   = str(random.randint(01, 12)).zfill(02)
    date_DD   = str(random.randint(01, 28)).zfill(02)
    date_sep  = '-'
    return  date_YYYY + date_MM + date_DD 

udfdate_rand = udf(date_rand, StringType())

In [37]:
leftjoin = leftjoin.withColumn("Date",lit(0))

In [38]:
leftjoin_w_date = leftjoin.withColumn("Date", udfdate_rand("Date"))

In [39]:
leftjoin_w_date.show()

+-------+-------+-------+----------------+----------+--------+
|Input 1|Input 2|Input 3|     Description|Day Period|    Date|
+-------+-------+-------+----------------+----------+--------+
|   Blue|      4| 1.2000|  DescriptionTWO|   Evening|19871022|
|   Blue|      5| 1.0000|  DescriptionONE|   Evening|19680926|
|  Green|      1| 1.3000|  DescriptionONE|     Night|19871022|
|  Green|      0| 1.4450|  DescriptionONE|     Night|19680926|
| Yellow|      7| 1.3250| DescriptionFOUR|   Morning|19610126|
|    Red|      5| 1.3000|DescriptionTHREE| Afternoon|19610126|
|    Red|      9| 1.4000|  DescriptionONE| Afternoon|19730510|
|    Red|      6| 1.7216|DescriptionTHREE| Afternoon|19651016|
+-------+-------+-------+----------------+----------+--------+



In [43]:
leftjoin_w_date.printSchema()

root
 |-- Input 1: string (nullable = true)
 |-- Input 2: string (nullable = true)
 |-- Input 3: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Day Period: string (nullable = true)
 |-- Date: string (nullable = true)



In [45]:
leftjoin_w_date = leftjoin_w_date.withColumn("Input 3",leftjoin_w_date["Input 3"].cast('float'))

In [46]:
leftjoin_w_date.printSchema()

root
 |-- Input 1: string (nullable = true)
 |-- Input 2: string (nullable = true)
 |-- Input 3: float (nullable = true)
 |-- Description: string (nullable = true)
 |-- Day Period: string (nullable = true)
 |-- Date: string (nullable = true)



In [52]:
df_calc2 =leftjoin_w_date.filter((leftjoin_w_date['Input 3'] < 1.31 ) 
                                 & (leftjoin_w_date['Input 1'] != 'Red') 
                                 & (leftjoin_w_date['Input 1'] != 'Green'))

In [50]:
df_calc2.show()

+-------+-------+-------+--------------+----------+--------+
|Input 1|Input 2|Input 3|   Description|Day Period|    Date|
+-------+-------+-------+--------------+----------+--------+
|   Blue|      4|    1.2|DescriptionTWO|   Evening|19830325|
|   Blue|      5|    1.0|DescriptionONE|   Evening|19760122|
+-------+-------+-------+--------------+----------+--------+



In [85]:
date_list = leftjoin_w_date.select('Date').collect()

date_array = [int(i.Date) for i in date_list]

In [86]:
type(date_array)

list

In [87]:
defg = sorted(date_array)
defg

[19610126,
 19610126,
 19651016,
 19680926,
 19680926,
 19730510,
 19871022,
 19871022]

In [88]:
len(defg)

8

In [89]:
if len(defg)%2 == 0:
    date_cmp1 = defg[(len(defg)/2)-1]
    date_cmp2 = defg[(len(defg)/2)]
    if date_cmp1 == date_cmp2:
        date_cmp = date_cmp1
    else:
        date_cmp = date_cmp1
else:
    date_cmp = defg[(len(defg)/2)-1]

In [90]:
date_cmp

19680926

In [91]:
import pyspark.sql.functions as F

In [92]:
leftjoin_w_date.printSchema()

root
 |-- Input 1: string (nullable = true)
 |-- Input 2: string (nullable = true)
 |-- Input 3: float (nullable = true)
 |-- Description: string (nullable = true)
 |-- Day Period: string (nullable = true)
 |-- Date: string (nullable = true)



In [107]:
leftjoin_w_date = leftjoin_w_date.withColumn("Date",leftjoin_w_date["Date"].cast('int'))
leftjoin_w_date = leftjoin_w_date.withColumn("Input 2",leftjoin_w_date["Input 2"].cast('int'))

In [119]:
leftjoin_w_date.printSchema()

root
 |-- Input 1: string (nullable = true)
 |-- Input 2: integer (nullable = true)
 |-- Input 3: float (nullable = true)
 |-- Description: string (nullable = true)
 |-- Day Period: string (nullable = true)
 |-- Date: integer (nullable = true)



In [ ]:
#leftjoin_w_flag = leftjoin_w_date.withColumn('flag', F.when((F.col("Date") > F.lit(str(date_cmp))) 
#                                                            & (F.col("Input 2") > F.lit('1')), 1).otherwise(0))

In [115]:
def flag_func(a,b):
    if a > date_cmp and b > 1:
        flag = 1 
    else:
        flag = 0 
    return flag

In [116]:
udf_falg_func = udf(flag_func)

In [117]:
leftjoin_w_flag = leftjoin_w_date.withColumn('flag', udf_falg_func("Date","Input 2"))

In [120]:
leftjoin_w_flag.show()

+-------+-------+-------+----------------+----------+--------+----+
|Input 1|Input 2|Input 3|     Description|Day Period|    Date|flag|
+-------+-------+-------+----------------+----------+--------+----+
|   Blue|      4|    1.2|  DescriptionTWO|   Evening|19871022|   1|
|   Blue|      5|    1.0|  DescriptionONE|   Evening|19680926|   0|
|  Green|      1|    1.3|  DescriptionONE|     Night|19610126|   0|
|  Green|      0|  1.445|  DescriptionONE|     Night|19730510|   0|
| Yellow|      7|  1.325| DescriptionFOUR|   Morning|19651016|   0|
|    Red|      5|    1.3|DescriptionTHREE| Afternoon|19830325|   1|
|    Red|      9|    1.4|  DescriptionONE| Afternoon|19760122|   1|
|    Red|      6| 1.7216|DescriptionTHREE| Afternoon|19591015|   0|
+-------+-------+-------+----------------+----------+--------+----+



In [124]:
df1 = leftjoin_w_flag.groupby('Description').agg(F.sum("Input 3")/(F.min("Input 2"))).alias('sumbymin')


In [125]:
df1.show()

+----------------+-----------------------------------------------------------------------------------------------+
|     Description|((sum(Input 3),mode=Complete,isDistinct=false) / (min(Input 2),mode=Complete,isDistinct=false))|
+----------------+-----------------------------------------------------------------------------------------------+
|  DescriptionONE|                                                                                           null|
|DescriptionTHREE|                                                                             0.6043200016021728|
| DescriptionFOUR|                                                                             0.1892857210976737|
|  DescriptionTWO|                                                                            0.30000001192092896|
+----------------+-----------------------------------------------------------------------------------------------+

